<a href="https://colab.research.google.com/github/alesaccoia/IULM_DDM2324_Notebooks/blob/main/18_BTYD_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilizzo della libreria "lifetimes" per il calcolo del valore del cliente

In [ ]:
!pip install lifetimes

E' possibile caricare il dataset originale con le transazioni di CD NOW

In [ ]:
from lifetimes.datasets import load_transaction_data
from lifetimes.utils import summary_data_from_transaction_data

transaction_data = load_transaction_data()
print(transaction_data.head())

"summary_data_from_transaction_data" crea il dataframe aggregato partendo dai dati raw

In [ ]:
summary_data_from_transaction_data(transaction_data, "id", "date")

In alternativa, la libreria già fornisce il dataset aggregato

In [ ]:
from lifetimes.datasets import load_cdnow_summary
data = load_cdnow_summary(index_col=[0])

print(data.head())

Vediamo ora la dimensionalitá del dataset:

In [ ]:
data.shape

In [ ]:
data.describe()

# Utilizzo del Modello

Il modello Pareto/NBD viene importanto con l'oggetto BetaGeoFitter, che ha un API simile ai modelli di sklearn

In [ ]:
from lifetimes import BetaGeoFitter

# similar API to scikit-learn and lifelines.
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(data['frequency'], data['recency'], data['T'])
print(bgf)

# La Probabilità degli utenti di essere ancora attivi

In [ ]:
prob_alive = bgf.conditional_probability_alive(data['frequency'], data['recency'], data['T'])
data['prob_alive'] = prob_alive
print(f"Il numero di clienti che sono ancora attivi é {data['prob_alive'].sum()} su {data.shape[0]}")

# La Matrice Frequenza/Tempo Trascorso

Una volta che il modello é stato caricato, si possono visualizzare altri grafici interessanti.

Considera il seguente scenario: un cliente ha acquistato da te ogni giorno per tre settimane di fila, e non abbiamo più sentito nulla da loro da mesi. Quali sono le probabilità che siano ancora "attivi"? Piuttosto basse.

D'altra parte, un cliente che storicamente acquista da te una volta al trimestre e ha acquistato l'ultimo trimestre, è probabile che sia ancora attivo.

Possiamo visualizzare questa relazione utilizzando la matrice Frequenza/Tempo trascorso, che calcola il numero previsto di transazioni che un cliente artificiale effettuerà nel prossimo periodo di tempo, in base alla sua recente attività (età all'ultimo acquisto) e alla frequenza (il numero di transazioni ripetute che ha effettuato).

In [ ]:
from lifetimes.plotting import plot_frequency_recency_matrix

plot_frequency_recency_matrix(bgf)


Possiamo notare che se un cliente ha acquistato 25 volte da te e il suo ultimo acquisto è stato quando avevano 35 settimane di età (considerando che l'individuo ha 35 settimane di età), allora sono il tuo miglior cliente (in basso a destra). I tuoi clienti meno attivi si trovano nell'angolo in alto a destra: hanno acquistato molto rapidamente e non li abbiamo visti da settimane.

C'è anche quella "coda" intorno a (5,25). Rappresenta il cliente che acquista raramente, ma l'abbiamo visto di recente, quindi potrebbero acquistare di nuovo - non siamo sicuri se siano inattivi o semplicemente tra un acquisto e l'altro.

Un'altra matrice interessante da esaminare è la probabilità di essere ancora attivi:

In [ ]:
from lifetimes.plotting import plot_probability_alive_matrix

plot_probability_alive_matrix(bgf)

## Calcolo della probabilitá di acquisto nel periodo seguente

In [ ]:
t = 1
data['predicted_purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(t, data['frequency'], data['recency'], data['T'])
data.head()

## Calcolo della probabilitá di acquisto nel periodo seguente (52 settimane)

In [ ]:
t = 52
data['predicted_purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(t,
                                                                                      data['frequency'],
                                                                                      data['recency'],
                                                                                      data['T'])

In [ ]:
print(data.head())
print(data.shape)

## Grafico della probabilità di un utente di essere ancora attivo

In [ ]:
from lifetimes.plotting import plot_history_alive

id = 35
days_since_birth = 200
sp_trans = transaction_data.loc[transaction_data['id'] == id]
plot_history_alive(bgf, days_since_birth, sp_trans, 'date')

# Stima del CLV
Carichiamo il dataset con anche il valore delle transazioni

In [ ]:
from lifetimes.datasets import load_cdnow_summary_data_with_monetary_value

summary_with_money_value = load_cdnow_summary_data_with_monetary_value()
summary_with_money_value.head()
returning_customers_summary = summary_with_money_value[summary_with_money_value['frequency']>0]

print(returning_customers_summary.head())

Il valore medio di una transazione di ritorno é:

In [ ]:
mean_monetary_value = returning_customers_summary['monetary_value'].mean()
mean_monetary_value

Possiamo quindi unire quest'informazione al nostro dataset, che contiene la probabilitá di acquisti nelle prossime 52 settimane, nel seguente modo

In [ ]:
data['predicted_purchases_amount'] = data['predicted_purchases'] * mean_monetary_value
CLV = data['predicted_purchases_amount'].sum() / data.shape[0]
print(CLV)

In [ ]:
id = 35
print(data.iloc[id])
